In [1]:
from ultralytics import YOLO
import torch
import copy

In [2]:
yolo_model_path = "../prerak/yolo11x.pt"

In [3]:
model = YOLO(yolo_model_path)

In [4]:
# Keep a copy of old state dict for sanity check
old_dict = copy.deepcopy(model.state_dict())

old_dict

OrderedDict([('model.model.0.conv.weight',
              tensor([[[[-3.5877e-03,  2.6343e-01, -2.5952e-01],
                        [-3.2387e-03, -2.7075e-01,  2.6489e-01],
                        [ 1.9817e-03,  4.6616e-03, -1.3294e-03]],
              
                       [[ 1.5778e-02,  1.7419e-01, -1.8152e-01],
                        [-1.2726e-02, -1.7004e-01,  1.7896e-01],
                        [-1.4439e-03, -2.3785e-03, -2.5749e-04]],
              
                       [[ 9.1629e-03,  2.3613e-03, -1.8539e-02],
                        [-3.3989e-03, -4.1313e-03,  1.2535e-02],
                        [-2.6855e-03, -1.5650e-03,  6.5613e-03]]],
              
              
                      [[[-2.3556e-03, -1.4734e-01, -2.3767e-01],
                        [ 9.2468e-03,  2.4902e-01,  4.3213e-01],
                        [-2.3079e-03, -1.0803e-01, -1.9946e-01]],
              
                       [[ 4.8409e-03,  7.4524e-02,  8.7646e-02],
                        [-6.1073

In [5]:
# We should freeze all but the last layer
model.state_dict().keys()

odict_keys(['model.model.0.conv.weight', 'model.model.0.bn.weight', 'model.model.0.bn.bias', 'model.model.0.bn.running_mean', 'model.model.0.bn.running_var', 'model.model.0.bn.num_batches_tracked', 'model.model.1.conv.weight', 'model.model.1.bn.weight', 'model.model.1.bn.bias', 'model.model.1.bn.running_mean', 'model.model.1.bn.running_var', 'model.model.1.bn.num_batches_tracked', 'model.model.2.cv1.conv.weight', 'model.model.2.cv1.bn.weight', 'model.model.2.cv1.bn.bias', 'model.model.2.cv1.bn.running_mean', 'model.model.2.cv1.bn.running_var', 'model.model.2.cv1.bn.num_batches_tracked', 'model.model.2.cv2.conv.weight', 'model.model.2.cv2.bn.weight', 'model.model.2.cv2.bn.bias', 'model.model.2.cv2.bn.running_mean', 'model.model.2.cv2.bn.running_var', 'model.model.2.cv2.bn.num_batches_tracked', 'model.model.2.m.0.cv1.conv.weight', 'model.model.2.m.0.cv1.bn.weight', 'model.model.2.m.0.cv1.bn.bias', 'model.model.2.m.0.cv1.bn.running_mean', 'model.model.2.m.0.cv1.bn.running_var', 'model.mod

In [6]:
# Add a callback to put the frozen layers in eval mode to prevent BN values
# from changing.
def put_in_eval_mode(trainer, n_layers=22):
  for i, (name, module) in enumerate(trainer.model.named_modules()):
    if name.endswith("bn") and int(name.split('.')[1]) < n_layers:
      module.eval()
      module.track_running_stats = False
      # print(name, " put in eval mode.")

In [7]:
model.add_callback("on_train_epoch_start", put_in_eval_mode)
model.add_callback("on_pretrain_routine_start", put_in_eval_mode)

In [8]:
num_gpus = torch.cuda.device_count()
print(f"Number of available GPUs: {num_gpus}")

Number of available GPUs: 2


In [9]:
yaml_path = "../prerak/yolo_dataset/dataset.yaml"
epochs = 50

In [10]:
results = model.train(data=yaml_path, epochs=epochs, freeze=23, imgsz=640)

Ultralytics 8.3.28 🚀 Python-3.12.2 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
engine/trainer: task=detect, mode=train, model=../prerak/yolo11x.pt, data=../prerak/yolo_dataset/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=23, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

train: Scanning /home2/harshit.aggarwal/prerak/yolo_dataset/train/labels.cache... 75 images, 0 backgrounds, 0 corrupt: 100%|██████████| 75/75 [00:00<?, ?it/s]

train: WARNING ⚠️ /home2/harshit.aggarwal/prerak/yolo_dataset/train/images/train_0015.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /home2/harshit.aggarwal/prerak/yolo_dataset/train/images/train_0054.jpg: corrupt JPEG restored and saved



val: Scanning /home2/harshit.aggarwal/prerak/yolo_dataset/val/labels.cache... 19 images, 0 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      3.23G     0.5335     0.3405     0.9596         12        640: 100%|██████████| 5/5 [00:01<00:00,  4.65it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.88it/s]

                   all         19         21      0.936      0.905      0.969      0.813



50 epochs completed in 0.051 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 114.4MB
Optimizer stripped from runs/detect/train/weights/best.pt, 114.4MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.28 🚀 Python-3.12.2 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
YOLO11x summary (fused): 464 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.51it/s]


                   all         19         21      0.905      0.952      0.978      0.836
Speed: 0.2ms preprocess, 11.7ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train


In [10]:
# train_args = {
#     'data': yaml_path,
#     'epochs': epochs,
#     'imgsz': 640,
#     'batch': 4,
#     'patience': 50,
#     'device': [i for i in range(num_gpus)],
#     'verbose': True,
#     'save': True,
#     'save_period': 10,  # Save more frequently
#     'workers': 8,
#     'exist_ok': True,
#     'pretrained': True,
#     'freeze': 23,  # Freeze all but the last layer
#     'optimizer': 'Adam',  # Adam works better for transfer learning
#     'lr0': 0.001,  # Lower initial learning rate
#     'lrf': 0.1,    # Final learning rate fraction
#     'momentum': 0.937,
#     'weight_decay': 0.0005,
#     'warmup_epochs': 3.0,
#     'warmup_momentum': 0.8,
#     'warmup_bias_lr': 0.1,
#     'box': 7.5,    # Box loss gain
#     'cls': 0.5,    # Class loss gain
#     'dfl': 1.5,    # DFL loss gain
#     'close_mosaic': 10,
#     'name': 'autorickshaw_detection_transfer_2',
#     'amp': True,   # Use automatic mixed precision
#     'cache': 'ram',
#     # Transfer learning specific augmentations
#     'augment': True,
#     'mosaic': 0.5,  # Reduce mosaic augmentation
#     'mixup': 0.25,  # Reduce mixup augmentation
#     'degrees': 10.0,
#     'translate': 0.1,
#     'scale': 0.5,
#     'shear': 2.0,
#     'fliplr': 0.5,
#     'hsv_h': 0.015,
#     'hsv_s': 0.7,
#     'hsv_v': 0.4,
# }

In [11]:
# results = model.train(**train_args)

Ultralytics 8.3.28 🚀 Python-3.12.2 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
engine/trainer: task=detect, mode=train, model=../prerak/yolo11x.pt, data=../prerak/yolo_dataset/dataset.yaml, epochs=100, time=None, patience=50, batch=4, imgsz=640, save=True, save_period=10, cache=ram, device=[0, 1], workers=8, project=None, name=autorickshaw_detection_transfer_2, exist_ok=True, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=22, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None

train: Scanning /home2/harshit.aggarwal/prerak/yolo_dataset/train/labels... 75 images, 0 backgrounds, 0 corrupt: 100%|██████████| 75/75 [00:01<00:00, 38.65it/s]


train: WARNING ⚠️ /home2/harshit.aggarwal/prerak/yolo_dataset/train/images/train_0015.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /home2/harshit.aggarwal/prerak/yolo_dataset/train/images/train_0054.jpg: corrupt JPEG restored and saved
train: New cache created: /home2/harshit.aggarwal/prerak/yolo_dataset/train/labels.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.1GB RAM): 100%|██████████| 75/75 [00:01<00:00, 66.70it/s]
val: Scanning /home2/harshit.aggarwal/prerak/yolo_dataset/val/labels... 19 images, 0 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<00:00, 53.80it/s]


val: New cache created: /home2/harshit.aggarwal/prerak/yolo_dataset/val/labels.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

                   all         19         21      0.944      0.857      0.935      0.683

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     99/100     0.977G     0.7066     0.4948      1.169          2        640: 100%|██████████| 19/19 [00:02<00:00,  7.67it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00, 16.60it/s]


                   all         19         21      0.943      0.857       0.93      0.681
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 49, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

99 epochs completed in 0.141 hours.
Optimizer stripped from runs/detect/autorickshaw_detection_transfer_2/weights/last.pt, 114.4MB
Optimizer stripped from runs/detect/autorickshaw_detection_transfer_2/weights/best.pt, 114.4MB

Validating runs/detect/autorickshaw_detection_transfer_2/weights/best.pt...
Ultralytics 8.3.9 🚀 Python-3.12.2 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                     CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
YOLO11x summary (fused): 464 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  5.20it/s]


                   all         19         21          1      0.945       0.98      0.738
Speed: 0.4ms preprocess, 44.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/autorickshaw_detection_transfer_2


In [11]:
def compare_dicts(state_dict1, state_dict2):
    # Compare the keys
    keys1 = set(state_dict1.keys())
    keys2 = set(state_dict2.keys())

    if keys1 != keys2:
        print("Models have different parameter names.")
        return False

    # Compare the values (weights)
    for key in keys1:
        if not torch.equal(state_dict1[key], state_dict2[key]):
            print(f"Weights for parameter '{key}' are different.")
            if "bn" in key and "22" not in key:
              state_dict1[key] = state_dict2[key]

# Compare the dicts. Changes should only be in layer 22 and above
compare_dicts(old_dict, model.state_dict())

Weights for parameter 'model.model.23.cv3.1.1.0.bn.weight' are different.
Weights for parameter 'model.model.23.cv3.2.0.0.bn.bias' are different.
Weights for parameter 'model.model.23.cv3.1.1.1.bn.running_var' are different.
Weights for parameter 'model.model.22.m.0.cv3.bn.running_var' are different.
Weights for parameter 'model.model.22.m.0.m.1.cv1.bn.running_mean' are different.
Weights for parameter 'model.model.22.m.1.cv3.bn.running_var' are different.
Weights for parameter 'model.model.23.cv2.1.1.bn.running_mean' are different.
Weights for parameter 'model.model.23.cv3.0.1.0.bn.running_var' are different.
Weights for parameter 'model.model.23.cv3.1.0.1.bn.bias' are different.
Weights for parameter 'model.model.22.m.1.cv2.bn.running_var' are different.
Weights for parameter 'model.model.23.cv3.1.0.0.bn.running_var' are different.
Weights for parameter 'model.model.23.cv3.1.0.1.bn.weight' are different.
Weights for parameter 'model.model.23.cv3.0.1.1.bn.running_mean' are different.


In [17]:
new_state_dict = dict()

#  Increment the head number by 1 in the state_dict
for k, v in model.state_dict().items():
  if k.startswith("model.model.23"):
    new_state_dict[k.replace("model.23", "model.24")] = v

# Save the current state_dict. Only layer 24.
torch.save(new_state_dict, "./runs/detect/train/weights/yolov11x_lp.pt")

### After all the changes to the model's config yaml file: cfg/models/11/yolo11.yaml

In [5]:
model_2 = YOLO('ultralytics/cfg/models/11/yolo11x.yaml', task='detect').load(yolo_model_path)

Transferred 1015/1136 items from pretrained weights


In [5]:
state_dict = torch.load("./runs/detect/train/weights/yolov11x_lp.pt")

In [6]:
# Load state_dict of the custom trained model head. Layer 23.
model_2.load_state_dict(state_dict, strict=False)

_IncompatibleKeys(missing_keys=['model.model.0.conv.weight', 'model.model.0.bn.weight', 'model.model.0.bn.bias', 'model.model.0.bn.running_mean', 'model.model.0.bn.running_var', 'model.model.1.conv.weight', 'model.model.1.bn.weight', 'model.model.1.bn.bias', 'model.model.1.bn.running_mean', 'model.model.1.bn.running_var', 'model.model.2.cv1.conv.weight', 'model.model.2.cv1.bn.weight', 'model.model.2.cv1.bn.bias', 'model.model.2.cv1.bn.running_mean', 'model.model.2.cv1.bn.running_var', 'model.model.2.cv2.conv.weight', 'model.model.2.cv2.bn.weight', 'model.model.2.cv2.bn.bias', 'model.model.2.cv2.bn.running_mean', 'model.model.2.cv2.bn.running_var', 'model.model.2.m.0.cv1.conv.weight', 'model.model.2.m.0.cv1.bn.weight', 'model.model.2.m.0.cv1.bn.bias', 'model.model.2.m.0.cv1.bn.running_mean', 'model.model.2.m.0.cv1.bn.running_var', 'model.model.2.m.0.cv2.conv.weight', 'model.model.2.m.0.cv2.bn.weight', 'model.model.2.m.0.cv2.bn.bias', 'model.model.2.m.0.cv2.bn.running_mean', 'model.model

In [7]:
image_path = "../prerak/archive/auto/auto/Datacluster Labs Auto (1).jpg"
image_path_2 = "../prerak/images/demo.png"

In [8]:
results = model_2.predict(image_path, save=True)


image 1/1 /home2/harshit.aggarwal/ultralytics/../prerak/archive/auto/auto/Datacluster Labs Auto (1).jpg: 320x640 1 1, 2 7s, 1 80, 56.6ms
Speed: 3.8ms preprocess, 56.6ms inference, 1.7ms postprocess per image at shape (1, 3, 320, 640)
Results saved to runs/detect/predict3


In [9]:
model_2.predict(image_path_2, save=True)


image 1/1 /home2/harshit.aggarwal/ultralytics/../prerak/images/demo.png: 384x640 5 0s, 2 2s, 5 3s, 1 9, 1 24, 74.4ms
Speed: 2.9ms preprocess, 74.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict3


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '10', 11: '11', 12: '12', 13: '13', 14: '14', 15: '15', 16: '16', 17: '17', 18: '18', 19: '19', 20: '20', 21: '21', 22: '22', 23: '23', 24: '24', 25: '25', 26: '26', 27: '27', 28: '28', 29: '29', 30: '30', 31: '31', 32: '32', 33: '33', 34: '34', 35: '35', 36: '36', 37: '37', 38: '38', 39: '39', 40: '40', 41: '41', 42: '42', 43: '43', 44: '44', 45: '45', 46: '46', 47: '47', 48: '48', 49: '49', 50: '50', 51: '51', 52: '52', 53: '53', 54: '54', 55: '55', 56: '56', 57: '57', 58: '58', 59: '59', 60: '60', 61: '61', 62: '62', 63: '63', 64: '64', 65: '65', 66: '66', 67: '67', 68: '68', 69: '69', 70: '70', 71: '71', 72: '72', 73: '73', 74: '74', 75: '75', 76: '76', 77: '77', 78: '78', 79: '79', 80: '80'}
 obb: None
 orig_img: array([[[25, 28, 25],
         [24, 27,

In [25]:
# save the weights of model_2
model_2.save("yolov11x_auto.pt")

In [31]:
torch.save(model_2.state_dict(), "yolov11x_auto_state_dict.pt")

In [10]:
def calculate_iou(box1, box2):
    """Calculate IoU between two boxes"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    return intersection / (area1 + area2 - intersection)

In [11]:
def filter_overlapping_detections(yolo_results, iou_threshold=0.8):
    """
    Filter YOLO results to remove detections that overlap with autorickshaws.
    
    Args:
        yolo_results: List of YOLO results, each containing boxes, classes, and ids
        iou_threshold: IoU threshold above which to remove overlapping detections
        
    Returns:
        Filtered YOLO results
    """
    def calculate_iou(box1, box2):
        # Calculate intersection coordinates
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        # Calculate intersection area
        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        
        # Calculate union area
        box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
        box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = box1_area + box2_area - intersection
        
        # Return IoU
        return intersection / union if union > 0 else 0

    filtered_results = []
    
    # Process each frame's results
    for result in yolo_results:
        boxes = result.boxes.xyxy.tolist()
        classes = result.boxes.cls.int().tolist()
        ids = result.boxes.id.int().tolist() if result.boxes.id is not None else [0] * len(boxes)
        
        # Check if any autorickshaws (class 80) are present
        auto_indices = [i for i, cls in enumerate(classes) if cls == 80]
        
        if len(auto_indices) == 0:
            filtered_results.append(result)
            continue
        
        indices_to_keep = set(range(len(boxes)))
        for auto_idx in auto_indices:
            auto_box = boxes[auto_idx]
            
            # Check overlap with all other boxes
            for i, box in enumerate(boxes):
                if i == auto_idx:
                    continue
                
                iou = calculate_iou(auto_box, box)
                # print iou details
                print(f"Auto box: {auto_box}, Box: {box}, IoU: {iou}, Class: {classes[i]}")
                if iou > iou_threshold:
                    indices_to_keep.discard(i)
        
        # Keep only non-overlapping detections
        boxes = [boxes[i] for i in indices_to_keep]
        classes = [classes[i] for i in indices_to_keep]
        ids = [ids[i] for i in indices_to_keep]
        
        # Create dictionary with filtered results
        frame_data = {
            "classes": classes,
            "ids": ids,
            "bboxes": boxes,
            "frame_path": str(result.path)
        }
        filtered_results.append(frame_data)
    
    return filtered_results

In [51]:
filtered_results = filter_overlapping_detections(results)

Auto box: [2192.611572265625, 316.2428283691406, 3837.9375, 1689.4727783203125], Box: [625.2488403320312, 90.50804901123047, 2350.12744140625, 1227.226806640625], IoU: 0.03519937939989239, Class: 7
Auto box: [2192.611572265625, 316.2428283691406, 3837.9375, 1689.4727783203125], Box: [2230.566650390625, 370.9816589355469, 3798.241943359375, 1656.44775390625], IoU: 0.8919110239328796, Class: 7
Auto box: [2192.611572265625, 316.2428283691406, 3837.9375, 1689.4727783203125], Box: [334.532958984375, 699.8336791992188, 721.1243286132812, 1032.3345947265625], IoU: 0.0, Class: 1


In [66]:
filtered_results

[{'classes': [7, 80, 1],
  'ids': [0, 0, 0],
  'bboxes': [[625.2488403320312,
    90.50804901123047,
    2350.12744140625,
    1227.226806640625],
   [2192.611572265625, 316.2428283691406, 3837.9375, 1689.4727783203125],
   [334.532958984375,
    699.8336791992188,
    721.1243286132812,
    1032.3345947265625]],
  'frame_path': '/home2/harshit.aggarwal/ultralytics/../prerak/archive/auto/auto/Datacluster Labs Auto (1).jpg'}]

In [4]:
from pathlib import Path
data = Path("../prerak/output_folder/cropped_frames/")

In [6]:
yolo_results = list(model_2.track(data, stream=True, persist=True, conf=0.7))


image 1/2640 /home2/harshit.aggarwal/ultralytics/../prerak/output_folder/cropped_frames/000000.jpg: 384x640 2 2s, 55.1ms
image 2/2640 /home2/harshit.aggarwal/ultralytics/../prerak/output_folder/cropped_frames/000001.jpg: 384x640 1 2, 23.3ms
image 3/2640 /home2/harshit.aggarwal/ultralytics/../prerak/output_folder/cropped_frames/000002.jpg: 384x640 1 2, 23.3ms
image 4/2640 /home2/harshit.aggarwal/ultralytics/../prerak/output_folder/cropped_frames/000003.jpg: 384x640 2 2s, 23.4ms
image 5/2640 /home2/harshit.aggarwal/ultralytics/../prerak/output_folder/cropped_frames/000004.jpg: 384x640 2 2s, 24.0ms
image 6/2640 /home2/harshit.aggarwal/ultralytics/../prerak/output_folder/cropped_frames/000005.jpg: 384x640 2 2s, 23.9ms
image 7/2640 /home2/harshit.aggarwal/ultralytics/../prerak/output_folder/cropped_frames/000006.jpg: 384x640 2 2s, 20.4ms
image 8/2640 /home2/harshit.aggarwal/ultralytics/../prerak/output_folder/cropped_frames/000007.jpg: 384x640 2 2s, 20.5ms
image 9/2640 /home2/harshit.aggar

In [16]:
import json
def save_yolo_results(yolo_results, output_file):
    data = []
    for result in yolo_results:
        frame_data = {}
        if result.boxes is None or len(result.boxes) == 0:
            frame_data = {
                "classes": [],
                "ids": [],
                "bboxes": [],
                "frame_path": str(result.path),
            }
        else:
            frame_data = {
                "classes": result.boxes.cls.int().tolist() if result.boxes.cls is not None else [],
                "ids": result.boxes.id.int().tolist() if result.boxes.id is not None else [],
                "bboxes": result.boxes.xyxy.tolist() if result.boxes.xyxy is not None else [],
                "frame_path": str(result.path),
            }
        data.append(frame_data)
    
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=2)

In [17]:
save_yolo_results(yolo_results, "../prerak/output_folder/track_output.json")